In [36]:
from flask import Flask, request, jsonify, redirect, url_for

import pickle
import json
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
fillna_vals = pd.read_csv('fillna_vals_red.csv')
display(fillna_vals)

,odor_a,odor_c,odor_f,odor_l,odor_m,odor_n,odor_p,odor_s,odor_y,gill-size_b,...,bruises_f,bruises_t,gill-spacing_c,gill-spacing_w,population_a,population_c,population_n,population_s,population_v,population_y
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [4]:
with open('feat_dicts.json') as json_file:
    feat_dicts = json.load(json_file) 

In [5]:
feat_dicts

{'cap-shape': {'b': 'bell',
  'c': 'conical',
  'x': 'convex',
  'f': 'flat',
  'k': 'knobbed',
  's': 'sunken'},
 'cap-surface': {'f': 'fibrous', 'g': 'grooves', 'y': 'scaly', 's': 'smooth'},
 'cap-color': {'n': 'brown',
  'b': 'buff',
  'c': 'cinnamon',
  'g': 'gray',
  'r': 'green',
  'p': 'pink',
  'u': 'purple',
  'e': 'red',
  'w': 'white',
  'y': 'yellow'},
 'bruises': {'t': 'bruises', 'f': 'no'},
 'odor': {'a': 'almond',
  'l': 'anise',
  'c': 'creosote',
  'y': 'fishy',
  'f': 'foul',
  'm': 'musty',
  'n': 'none',
  'p': 'pungent',
  's': 'spicy'},
 'gill-attachment': {'a': 'attached',
  'd': 'descending',
  'f': 'free',
  'n': 'notched'},
 'gill-spacing': {'c': 'close', 'w': 'crowded', 'd': 'distant'},
 'gill-size': {'b': 'broad', 'n': 'narrow'},
 'gill-color': {'k': 'black',
  'n': 'brown',
  'b': 'buff',
  'h': 'chocolate',
  'g': 'gray',
  'r': 'green',
  'o': 'orange',
  'p': 'pink',
  'u': 'purple',
  'e': 'red',
  'w': 'white',
  'y': 'yellow'},
 'stalk-shape': {'e': '

In [42]:
kt = tuple(feat_dicts.keys())

In [45]:
feat_dicts['ring-type']

{'c': 'cobwebby',
 'e': 'evanescent',
 'f': 'flaring',
 'l': 'large',
 'n': 'none',
 'p': 'pendant',
 's': 'sheathing',
 'z': 'zone'}

In [7]:
fillna_vals['odor_f'].values[0]

0.0

In [8]:
url = 'http://127.0.0.1:5000/mushroom/?'
for c in fillna_vals.columns:
    if fillna_vals[c].values[0]==1:
        s = str(c).split(sep='_')
        url += s[0]
        url += "="
        url += s[1]
        url += "&"
url = url[:-1]

url
    

'http://127.0.0.1:5000/mushroom/?odor=p&gill-size=n&ring-type=p&stalk-surface-below-ring=s&stalk-surface-above-ring=s&spore-print-color=k&gill-color=k&bruises=t&gill-spacing=c&population=s'

In [9]:
important_cols = ['odor', 'gill-size', 'ring-type', 'stalk-surface-below-ring','stalk-surface-above-ring', 'spore-print-color', 'gill-color','bruises', 'gill-spacing', 'population']

In [29]:
def build_static_form(feat_dict):
  action = 'process_form'
  html =  f'<body>\n\
    <h1>Choose features values</h1>\n\
    <form action="{action}" method="POST">\n\
      <label for="name">MushroomFeatures</label>\n\
        '

  for c in important_cols:

      html += f'  <fieldset>\n\
            <legend>{str(c)}</legend>\n\
            '
      for i, (k, v) in enumerate(feat_dicts[c].items()):

        html += f'<div>\n\
                <input type="radio" id={str(c)+"_"+str(k)} name={str(c)} value={str(k)} checked>\n\
                <label for={str(k)}>{str(v)}</label>\n\
            </div>\n\
          \
    '
      html +='\n\
          </fieldset>\n\
    '
  html += '<input type="submit" value="Submit" id="submit">\n'
  html += '</form>\n\
  </body>'
  return html

In [40]:
app = Flask(__name__)

model_from_disk = pickle.load(open('model_red.pkl','rb'))
ohe_from_disk = pickle.load(open('ohe_red.pkl','rb'))

@app.route('/')
def home():
    #txt = f'Welcome \
    #    Example url: {url} \
    #    '
    return build_static_form(feat_dicts)

@app.route('/mushroom/')
def give_mushroom_predict():
    
    a = []
    for c in important_cols:
        a.append(request.args.get(str(c)))
    
    # faire un dataframe avec la requête
    raw_input = pd.DataFrame(np.array(a).reshape(1,-1))
    raw_input.columns = important_cols

    # traduire le dataframe selon onehotoncoder sur disque
    model_cols = model_from_disk.feature_names_in_
    new_data = pd.DataFrame(ohe_from_disk.transform(raw_input))
    new_data.columns = model_cols

    # faire une prediction
    #pred = model_from_disk.predict(fillna_vals)
    pred = model_from_disk.predict(new_data)
    
    # mettre en json le resultat de la prediction
    dict_pred = {'class':pred[0]}
    json_object = json.dumps(dict_pred, indent = 4) 

    return jsonify(dict_pred)

@app.route('/process_form', methods=['POST'])
def process_form():
    
    params = {}
    for c in important_cols:
        params[c] = request.form[c]

    return redirect(url_for("predict", **params))

@app.route("/predict")
def predict():

    a = []
    for c in important_cols:
        a.append(request.args.get(str(c)))
    
    # faire un dataframe avec la requête
    raw_input = pd.DataFrame(np.array(a).reshape(1,-1))
    raw_input.columns = important_cols

    # traduire le dataframe selon onehotoncoder sur disque
    model_cols = model_from_disk.feature_names_in_
    new_data = pd.DataFrame(ohe_from_disk.transform(raw_input))
    new_data.columns = model_cols

    # faire une prediction
    #pred = model_from_disk.predict(fillna_vals)
    pred = model_from_disk.predict(new_data)
    
    # mettre en json le resultat de la prediction
    dict_pred = {'class':pred[0]}
    json_object = json.dumps(dict_pred, indent = 4) 

    return jsonify(dict_pred)
    

In [46]:
if __name__ == '__main__':

    app.run(port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Jul/2023 16:52:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Jul/2023 16:53:09] "POST /process_form HTTP/1.1" 302 -
127.0.0.1 - - [30/Jul/2023 16:53:09] "GET /predict?odor=c&gill-size=b&ring-type=l&stalk-surface-below-ring=y&stalk-surface-above-ring=s&spore-print-color=o&gill-color=h&bruises=f&gill-spacing=w&population=n HTTP/1.1" 200 -
